# Dataset cleaning

This notebook implements de-deduplication using the pandas library and advanced filtering using Cleanlab.

#### Step 0: Load dependencies

In [ ]:
import os
import pandas as pd
from cleanlab_tlm import TLM
import time

### Step 1: Filter EXACT duplicates

This removes duplicates using pandas. Reduces the load for Cleanlab later.

In [ ]:
# Config
INPUT_FILE = "prompt_responses_normalized.csv"
OUTPUT_FILE = "prompt_responses_deduplicated.csv"
COLUMN_TO_DEDUPLICATE = "prompt"

# Loads a CSV, removes duplicate rows based on a specific column, and saves the result to a new file.
def deduplicate_csv(filepath, column_name):
    print("--- Starting Deduplication Script ---")
    
    # 1. Check if the input file exists
    if not os.path.exists(filepath):
        print(f"FATAL ERROR: Input file not found at '{filepath}'")
        return

    try:
        # 2. Load the CSV into a pandas DataFrame
        print(f"Loading data from '{filepath}'...")
        df = pd.read_csv(filepath)
        original_row_count = len(df)
        print(f"Successfully loaded {original_row_count} rows.")

        # 3. Check if the specified column exists
        if column_name not in df.columns:
            print(f"FATAL ERROR: Column '{column_name}' not found in the CSV file.")
            return

        # 4. Perform the deduplication
        print(f"Removing duplicate rows based on the '{column_name}' column...")
        # - subset=[column_name]: Specifies that only this column should be checked for duplicates.
        # - keep='first': Keeps the first occurrence of a duplicate and removes the rest.
        deduplicated_df = df.drop_duplicates(subset=[column_name], keep='first')
        new_row_count = len(deduplicated_df)
        
        # 5. Report the results
        rows_removed = original_row_count - new_row_count
        print(f"\n--- Results ---")
        print(f"Original row count: {original_row_count}")
        print(f"Rows removed: {rows_removed}")
        print(f"Final row count: {new_row_count}")
        print("---------------")

        # 6. Save the cleaned DataFrame to a new CSV file
        print(f"Saving deduplicated data to '{OUTPUT_FILE}'...")
        deduplicated_df.to_csv(OUTPUT_FILE, index=False)
        print("Script finished successfully.")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    deduplicate_csv(INPUT_FILE, COLUMN_TO_DEDUPLICATE)


### Step 2: Adding Bias and Toxicity columns with Cleanlab Studios

This script adds bias_score and toxic_score based on either the 'prompt' column or 'response column' to prompt_responses_deduplicated.csv using Cleanlab Studio’s Python API. It outputs the result as either response_with_bias_toxic.xsc or prompt_with_bias_toxic.csv. 

In [ ]:
"""
Parameters
----------
api_key          : Cleanlab Studio API key.
dataset_name     : Name of the dataset already uploaded in Studio.
text_column      : Column to analyze ("prompt" or "response").
raw_csv_path     : Path to the original CSV on disk.
output_csv_path  : Where to save the enriched CSV.
project_name     : Studio project name.
poll_interval    : Seconds between status polls.
"""
def add_bias_toxic_scores(
    api_key: str,
    dataset_name: str,
    text_column: str,
    raw_csv_path: str,
    output_csv_path: str,
    project_name: str = "Bias & Toxicity scan",
    poll_interval: int = 60,
):
    studio = Studio(api_key)

    # 1. Resolve dataset → ID
    dataset_id = studio.poll_dataset_id_for_name(dataset_name)

    # 2. Kick off the project
    project_id = studio.create_project(
        dataset_id=dataset_id,
        project_name=project_name,
        modality="text",
        task_type="unsupervised",
        model_type="regular",
        text_column=text_column,
    )

    # 3. Wait until a cleanset exists
    while True:
        try:
            cleanset_id = studio.get_latest_cleanset_id(project_id)
            break
        except ValueError:
            time.sleep(poll_interval)

    studio.wait_until_cleanset_ready(cleanset_id)

    # 4. Download scores
    cl_cols = studio.download_cleanlab_columns(
        cleanset_id, include_cleanlab_columns=True
    )
    scores_only = cl_cols[["cleanlab_row_ID", "bias_score", "toxic_score"]]

    # 5. Merge back to local CSV and save
    raw = pd.read_csv(raw_csv_path)
    merged = raw.merge(scores_only, left_index=True, right_on="cleanlab_row_ID")
    merged.to_csv(output_csv_path, index=False)
    print(f"✅  New file written: {output_csv_path}")

    return merged

In [ ]:
# Run for "prompt" column
add_bias_toxic_scores(
    api_key= "YOUR_API_KEY",
    dataset_name= "prompt_responses_deduplicated.csv",
    text_column= "prompt",  
    raw_csv_path= "prompt_responses_deduplicated.csv",
    output_csv_path= "prompt_with_bias_toxic.csv",
)

# Run for "response" column
add_bias_toxic_scores(
    api_key= "YOUR_API_KEY",
    dataset_name= "prompt_responses_deduplicated.csv",
    text_column= "response",
    raw_csv_path= "response_responses_deduplicated.csv",
    output_csv_path= "response_with_bias_toxic.csv",
)

### Step 3: Filter by bias_score and toxic_score

Based on the scores, we can easily filter out low-quality, toxic, or unsafe content. A high score is a strong indicator of a problematic response. This cell does not actually remove rows or write to a new file, it just outputs the rows over either threshold. 

In [ ]:
# Config
INPUT_FILE = "response_with_bias_toxic.csv"  # The file from Cleanlab Studio with all the scores
BIAS_THRESHOLD = 0.75
TOXIC_THRESHOLD = 0.75

def filter_by_scores(filepath, bias_thresh, toxic_thresh):
    # 1. Check if the input file exists
    if not os.path.exists(filepath):
        print(f"FATAL ERROR: Input file not found at '{filepath}'")
        return

    try:
        # 2. Load the CSV into a pandas DataFrame
        print(f"Loading data from '{filepath}'...")
        df = pd.read_csv(filepath)
        original_row_count = len(df)
        print(f"Successfully loaded {original_row_count} rows.")

        # 3. Check if the required score columns exist
        bias_col = 'cleanlab_bias_score'
        toxic_col = 'cleanlab_toxic_score'
        
        if bias_col not in df.columns or toxic_col not in df.columns:
            print(f"FATAL ERROR: Required columns '{bias_col}' and/or '{toxic_col}' not found.")
            print("Please check the exact column names in your CSV file and update the script if needed.")
            return

        # 4. Identify rows to be removed
        condition_to_flag = (df[bias_col] >= bias_thresh) | (df[toxic_col] >= toxic_thresh)
        response_rows_to_flag_df = df[condition_to_flag]
        response_flagged_count = len(response_rows_to_flag_df)
        
        # 5. Report the results
        print(f"\n--- Results ---")
        print(f"Filtering with bias_score >= {bias_thresh} OR toxic_score >= {toxic_thresh}")
        print(f"Num rows flagged: {len(response_rows_to_flag_df)}")

        pd.set_option('display.max_colwidth', None)
            
    except Exception as e:
        print(f"An error occurred: {e}")

    flagged_rows = response_rows_to_flag_df[['prompt', 'response', "cleanlab_row_ID", bias_col, toxic_col]]

    return flagged_rows

In [ ]:
# Return rows with bias score > 0.75 or toxic_score > 0.75 based on the column 'response'
response_flagged_rows = filter_by_scores("response_with_bias_toxic.csv", BIAS_THRESHOLD, TOXIC_THRESHOLD)

# Return rows with bias score > 0.75 or toxic_score > 0.75 based on the column 'prompt'
prompt_flagged_rows = filter_by_scores("prompt_with_bias_toxic.csv" , BIAS_THRESHOLD, TOXIC_THRESHOLD)

### Step 4: Combine the two dataframes into one

Combine the rows flagged with high toxic_score and bias_score from the Cleanlab analysis based on the "prompt" column and from the Cleanlab analysis based on the "response" column

In [ ]:
# Tag each DataFrame’s score columns
prompt_df   = prompt_flagged_rows.rename(
    columns={
        "cleanlab_bias_score":   "prompt_bias_score",
        "cleanlab_toxic_score":  "prompt_toxic_score",
    }
)

response_df = response_flagged_rows.rename(
    columns={
        "cleanlab_bias_score":   "response_bias_score",
        "cleanlab_toxic_score":  "response_toxic_score",
    }
)

# Merge on the row identifier
combined_flagged_rows = (
    pd.merge(prompt_df, response_df,
             on="cleanlab_row_ID",
             how="outer",
             suffixes=("_prompt", "_response"))
      .sort_values("cleanlab_row_ID")
      .reset_index(drop=True)
)

# Fill missing prompt/response values
combined_flagged_rows["prompt"] = combined_flagged_rows["prompt_prompt"].combine_first(combined_flagged_rows["prompt_response"])
combined_flagged_rows["response"] = combined_flagged_rows["response_response"].combine_first(combined_flagged_rows["response_prompt"])

# Replace NaNs with 0 for easier filtering
combined_flagged_rows.fillna(
    {"prompt_bias_score":   0, "prompt_toxic_score":   0,
     "response_bias_score": 0, "response_toxic_score": 0},
    inplace=True
)

# Drop redundant columns
combined_flagged_rows = combined_flagged_rows[[
    "prompt", "response", "cleanlab_row_ID",
    "prompt_bias_score", "prompt_toxic_score",
    "response_bias_score", "response_toxic_score"
]]

# display(combined_flagged_rows)

# Export to CSV
combined_flagged_rows.to_csv("flagged_bias_toxic_rows.csv", index=False)

### Step 5: Remove near-duplicate columns

The final labelled dataset from Cleanlabs (analyzed on 'prompt') was saved (10,000 rows) with additional columns. They are:

- PII
- Toxic Score
- Bias Score
- Near Duplicate Cluster ID

This script removes near‑duplicates and keeps only one row per cluster ID.

- Near Duplicate Cluster is NaN for unique rows; non‑zero for rows that belong to a duplicate cluster.  
- For every cluster ID > 0, we keep the first row encountered and drop the rest.  
- Finally, we drop the Near Duplicate Cluster ID column in the final file.


In [ ]:
# config
INPUT_FILE   = "cleanlab_PROMPT-Bias-&-Toxicity-scan_2025-07-25-14-52-52.csv"
OUTPUT_FILE  = "cleaned_prompt_response.csv"
CLUSTER_COL  = "cleanlab_near_duplicate_cluster_id"

# 1) Load the CSV
df = pd.read_csv(INPUT_FILE)

# 2) Make sure the cluster column is numeric (NaN for blanks)
cluster = pd.to_numeric(df[CLUSTER_COL], errors="coerce")

# 3) Identify extra duplicates:
#    - rows whose cluster ID is not‑NaN
#    - AND are *not* the first occurrence of that cluster
extra_dups = (cluster.notna() &
              df.duplicated(subset=CLUSTER_COL, keep="first"))

# 4) Drop the extras and the cluster column
df_clean = (df[~extra_dups]
            .drop(columns=[CLUSTER_COL])
            .reset_index(drop=True))

# 5) Save
df_clean.to_csv(OUTPUT_FILE, index=False)
print(f"{OUTPUT_FILE} written. {extra_dups.sum()} rows removed.")
print(f"Final row count: {len(df_clean)}")

### Step 6: Repeat everything for the 246 rows that weren't added to Cleanlab because of the 10,000 row limit

In [ ]:
# Create a new dataset for the remaining 246 rows.
df = pd.read_csv("prompt_responses_deduplicated.csv")
last_246 = df.tail(246)

# display(last_246.head())

last_246.to_csv("last_246_df.csv", index=False)

# Upload to Cleanlab before proceeding

In [ ]:
# Cleanlab analysis on "response" column
add_bias_toxic_scores(
    api_key= "YOUR_API_KEY",
    dataset_name= "prompt_responses_deduplicated.csv",
    text_column= "response",
    raw_csv_path= "response_responses_deduplicated.csv",
    output_csv_path= "246_response_with_bias_toxic.csv",
)

In [ ]:
# Cleanlab analysis on "prompt" column
add_bias_toxic_scores(
    api_key= "YOUR_API_KEY",
    dataset_name= "last_246_df.csv",
    text_column= "prompt",
    raw_csv_path= "last_246_df.csv",
    output_csv_path= "246_prompt_with_bias_toxic.csv",
)

In [ ]:
# Filter by threshold
flagged_prompt_rows_246 = filter_by_scores("246_prompt_with_bias_toxic.csv", BIAS_THRESHOLD, TOXIC_THRESHOLD)

In [ ]:
flagged_response_rows_246 = filter_by_scores("246_response_with_bias_toxic.csv", BIAS_THRESHOLD, TOXIC_THRESHOLD)

In [ ]:
display(flagged_response_rows_246)

In [ ]:
# Combine the two dataframes

# Tag each DataFrame’s score columns
prompt_df   = flagged_prompt_rows_246.rename(
    columns={
        "cleanlab_bias_score":   "prompt_bias_score",
        "cleanlab_toxic_score":  "prompt_toxic_score",
    }
)

response_df = flagged_response_rows_246.rename(
    columns={
        "cleanlab_bias_score":   "response_bias_score",
        "cleanlab_toxic_score":  "response_toxic_score",
    }
)

# Merge on the row identifier
combined_flagged_rows_246 = (
    pd.merge(prompt_df, response_df,
             on="cleanlab_row_ID",      # common key
             how="outer")               # union of both sets
      .sort_values("cleanlab_row_ID")
      .reset_index(drop=True)
)

# Replace NaNs with 0 for easier filtering
combined_flagged_rows_246.fillna(
    {"prompt_bias_score":   0, "prompt_toxic_score":   0,
     "response_bias_score": 0, "response_toxic_score": 0},
    inplace=True
)

display(combined_flagged_rows_246[['prompt_x', 'response_y', 'prompt_bias_score', 'prompt_toxic_score', 'response_bias_score', 'response_toxic_score']])

In [ ]:
# Append combined_flagged_rows_246 to flagged_bias_toxic_rows.csv, removing duplicates
master_csv = "flagged_bias_toxic_rows.csv"

try:
    master_df = pd.read_csv(master_csv)
except FileNotFoundError:
    master_df = pd.DataFrame(columns=combined_flagged_rows_246.columns)

# Concatenate and deduplicate
updated_df = pd.concat([master_df, combined_flagged_rows_246], ignore_index=True)
updated_df = updated_df.drop_duplicates(subset=["cleanlab_row_ID"], keep="first").reset_index(drop=True)

# Drop the column cleanlab_row_ID because it's not needed anymore
updated_df = updated_df.drop(columns=["cleanlab_row_ID"])

# Save back to disk
updated_df.to_csv(master_csv, index=False)
print(f"flagged_bias_toxic_rows.csv updated. Total rows: {len(updated_df)}")

In [ ]:
# Near deduplication for last_246_df.csv

# Config
INPUT_FILE   = "cleanlab_246_prompt_with_bias_toxic_2025-07-25-19-42-47.csv"
OUTPUT_FILE  = "246_cleaned_prompt_response.csv"
CLUSTER_COL  = "cleanlab_near_duplicate_cluster_id"

# Load the CSV
df = pd.read_csv(INPUT_FILE)

# Make sure the cluster column is numeric (NaN for blanks)
cluster = pd.to_numeric(df[CLUSTER_COL], errors="coerce")

# Identify extra duplicates:
#    - rows whose cluster ID is not‑NaN
#    - AND are *not* the first occurrence of that cluster
extra_dups = (cluster.notna() &
              df.duplicated(subset=CLUSTER_COL, keep="first"))


# Drop the extras and the cluster column
df_clean = (df[~extra_dups]
            .drop(columns=[CLUSTER_COL])
            .reset_index(drop=True))

# Save
df_clean.to_csv(OUTPUT_FILE, index=False)
print(f"{OUTPUT_FILE} written. {extra_dups.sum()} rows removed.")
print(f"Final row count: {len(df_clean)}")

In [ ]:
# Add 246_cleaned_prompt_response.csv to the end of cleaned_prompt_response.csv
df_clean_246 = pd.read_csv("246_cleaned_prompt_response.csv")
df_clean = pd.read_csv("cleaned_prompt_response.csv")

df_combined = pd.concat([df_clean_246, df_clean], ignore_index=True).drop(columns=['cleanlab_row_ID']).reset_index(drop=True)
# Display the combined DataFrame
# display(df_combined.head())

df_combined.to_csv("dataset_final.csv", index=False)

print(len(df_combined))